# Replace missing values with averages
#### Replace missing iages with subgroup averages entirely in the database. This greatly reduces network traffic and latencies between the client Python code and the remote database server.

In [ ]:
import pandas as pd
from pandas import DataFrame
from DATA225utils import make_connection, dataframe_query

In [ ]:
conn = make_connection(config_file = 'titanic.ini')
cursor = conn.cursor()

## Age counts by class
#### In a dataframe, show the counts of missing ages and their percentages by class.

In [ ]:
def print_age_counts_by_class():
    _, df = dataframe_query(conn, 
        """
        SELECT class, 
               COUNT(*) AS class_count,
               COUNT(IF (age = 0, 1, NULL)) AS NA_count,
               100*COUNT(IF (age = 0, 1, NULL))/COUNT(*) AS NA_pct
        FROM passengers
        GROUP BY class
        ORDER BY class
        """
                           )

    display(df)

In [ ]:
print('\n% MISSING AGES BY CLASS')
print_age_counts_by_class()

## Age counts by sex by class
#### In a dataframe, show the counts of missing ages and their percentages by sex by class.

In [ ]:
def print_age_counts_by_sex():
    _, df = dataframe_query(conn, 
        """
        SELECT class,
               COUNT(IF (sex = 'female', 1, NULL)) AS female_count,
               COUNT(IF (sex = 'female' AND age = 0, 1, NULL)) AS female_NA_count,
               100*COUNT(IF (age = 0 AND sex = 'female', 1, NULL))
                  /COUNT(IF (sex = 'female', 1, NULL)) AS female_NA_pct,
               COUNT(IF (sex = 'male', 1, NULL)) AS male_count,
               COUNT(IF (sex = 'male' AND age = 0, 1, NULL)) AS male_NA_count,
               100*COUNT(IF (age = 0 AND sex = 'male', 1, NULL))
                  /COUNT(IF (sex = 'male', 1, NULL)) AS male_NA_pct
        FROM passengers
        GROUP BY class
        ORDER BY class
        """
                           )

    display(df)

In [ ]:
print('\n% MISSING AGES BY SEX BY CLASS')
print_age_counts_by_sex()

## Age counts by survived by sex by class
#### In a dataframe, show the counts of missing ages and their percentages by surveved by sex by class.

In [ ]:
def print_age_counts_by_survived():
    _, df = dataframe_query(conn, 
        """
        SELECT class,
               survived,
               COUNT(IF (sex = 'female', 1, NULL)) AS female_count,
               COUNT(IF (sex = 'female' AND age = 0, 1, NULL)) AS female_NA_count,
               100*COUNT(IF (age = 0 AND sex = 'female', 1, NULL))
                  /COUNT(IF (sex = 'female', 1, NULL)) AS female_NA_pct,
               COUNT(IF (sex = 'male', 1, NULL)) AS male_count,
               COUNT(IF (sex = 'male' AND age = 0, 1, NULL)) AS male_NA_count,
               100*COUNT(IF (age = 0 AND sex = 'male', 1, NULL))
                  /COUNT(IF (sex = 'male', 1, NULL)) AS male_NA_pct
        FROM passengers
        GROUP BY class, survived
        ORDER BY class, survived
        """
                           )

    display(df)

In [ ]:
print('\n% MISSING AGES BY SURVIVAL BY SEX BY CLASS')
print_age_counts_by_survived()

## Average age in each class

In [ ]:
def print_average_age_class():
    _, df = dataframe_query(conn, 
        """
        SELECT class, AVG(age)
        FROM passengers
        WHERE age > 0
        GROUP BY class
        ORDER BY class
        """
                           )

    display(df)

## A stored function to return average ages
#### Given a passenger class, sex of the passenger, and survival status, return the average age of the passengers in that class. A `DETERMINISTIC` function always returns the same result for the same parameters values.

In [ ]:
def create_stored_function():
    cursor.execute('DROP FUNCTION IF EXISTS average_age')

    sql = ( """
        CREATE FUNCTION average_age(class_parm VARCHAR(3), 
                                    sex_parm VARCHAR(6), 
                                    survived_parm VARCHAR(3))
            RETURNS DOUBLE
            DETERMINISTIC

        BEGIN
            DECLARE average DOUBLE;

            SET average = (
                SELECT AVG(age) AS average
                FROM passengers
                WHERE age > 0
                AND   class    = class_parm
                AND   sex      = sex_parm
                AND   survived = survived_parm
                GROUP BY class, sex, survived
            );

            RETURN average;
        END
        """
          )

    cursor.execute(sql)

In [ ]:
create_stored_function()

## Average age of each survived subgroup

In [ ]:
def print_average_age_survived():
    table = []

    for klass in ['1st', '2nd', '3rd']:
        row = [klass]

        cursor.execute(f"SELECT average_age('{klass}', 'female', 'no')")
        row.append(cursor.fetchone()[0])

        cursor.execute(f"SELECT average_age('{klass}', 'male', 'no')")
        row.append(cursor.fetchone()[0])

        cursor.execute(f"SELECT average_age('{klass}', 'female', 'yes')")
        row.append(cursor.fetchone()[0])

        cursor.execute(f"SELECT average_age('{klass}', 'male', 'yes')")
        row.append(cursor.fetchone()[0])

        table.append(row)

    df = DataFrame(table)
    df.columns=['class', 'female perished', 'male perished', 
                         'female survived', 'male survived']

    display(df)

## Replace the missing ages in each survived subgroup

In [ ]:
def replace_missing_ages():
    cursor.execute('SET SQL_SAFE_UPDATES = 0')

    cursor.execute( 
        """
        UPDATE passengers
        SET age = average_age(class, sex, survived)
        WHERE age = 0
        """
                  )

    conn.commit()

## Main

In [ ]:
print('\n% MISSING AGES BY CLASS')
print_age_counts_by_class()

print('\n% MISSING AGES BY SEX BY CLASS')
print_age_counts_by_sex()

print('\n% MISSING AGES BY SURVIVAL BY SEX BY CLASS')
print_age_counts_by_survived()

create_stored_function()

print('\nAVERAGE AGES BY SURVIVED BY SEX BY CLASS'
      '\nWITHOUT MISSING AGES')    
print_average_age_survived()

replace_missing_ages()

print('\nAVERAGE AGES BY SURVIVED BY SEX BY CLASS'
      '\nAFTER REPLACEMENTS WITH SURVIVED SUBGROUP AVERAGES')
print_average_age_class()

In [ ]:
cursor.close()
conn.close()

#### (c) 2023 by Ronald Mak